In [2]:
import re
import pandas as pd
import os
import itertools
import json
import sklearn
import evaluate

/home/michele.maggini/miniconda3/envs/bert_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
###Dataset###
tsv_directory = '/home/michele.maggini/dataset/CLEF_1C/CLEF_1c'
filename = 'CLEF_1C_Bulgarian.tsv'
df_path = os.path.join(tsv_directory, filename)
df_gt = pd.read_csv(df_path, sep="\t", engine="python")

cd = '/home/michele.maggini/Llama3/'
filename = 'CLEF1C_Bulgarian.csv'
path = os.path.join(cd, filename)
df_ans = pd.read_csv(path)

In [4]:
df_gt

,tweet_text,class_label
0,"За тия, дет си мислят, че няма вирус https://t...",0
1,"Отменят се част от противоепидемичните мерки, ...",0
2,Oпазването на биоразнообразието 🦋🐅🐘🌴 остава пр...,0
3,Кратък обзор над мерките в сградостроителствот...,0
4,Нова платформа за споделяне на добри практики ...,0
...,...,...
3688,"Два случая с COVID-19 в СБАЛББ Троян, един в Л...",1
3689,Генералният секретар на Съвета на Европа Мария...,0
3690,Нов потвърден случай на Covid-19 в Шипково htt...,1
3691,"@Anne_Hidalgo @Paris И, въпреки това, вчера, з...",0


In [5]:
for index, row in df_ans.iterrows():
    print(row)

0    0\n\nThe tweet appears to be a legitimate warn...
Name: 0, dtype: object
0    1\n\nReasoning:\n\n* Key words and phrases: "О...
Name: 1, dtype: object
0    0\n\nThe tweet appears to be a genuine and inf...
Name: 2, dtype: object
0    0\n\nThe tweet appears to be a factual and inf...
Name: 3, dtype: object
0    0\n\nThe tweet appears to be a genuine and inf...
Name: 4, dtype: object
0    1\n\nReasoning:\n\n* Key words and phrases: "8...
Name: 5, dtype: object
0    0\n\nThe tweet appears to be a legitimate and ...
Name: 6, dtype: object
0    0\n\nThe tweet appears to be a genuine and pos...
Name: 7, dtype: object
0    0\n\nThe given tweet appears to be a philosoph...
Name: 8, dtype: object
0    0\n\nThe tweet appears to be a factual report ...
Name: 9, dtype: object
0    1\n\nReasoning:\n\n* Key words and phrases: "C...
Name: 10, dtype: object
0    0\n\nThe tweet appears to be a promotional twe...
Name: 11, dtype: object
0    0\n\nThe tweet appears to be a legitimate and ...
Name: 1

In [6]:
def extract_last_line(text):
    # Split the text into lines
    lines = text.strip().split('\n')
    # Return the last line
    return lines[-1]

In [7]:
listLastline = df_ans['0'].apply(lambda x: extract_last_line(x))
df_lastline = listLastline.to_list()
len(df_lastline)

3693

In [8]:
def remove_keywords(input_list):
    # Define the keywords to be removed
    keywords = ["(True)", "(true)", "(False)", "(false)"]
    
    # Function to remove keywords from a single string
    def remove_from_string(s):
        for keyword in keywords:
            s = s.replace(keyword, "")
        return s.strip()  # Remove leading/trailing whitespace
    
    # Apply the function to each element in the list
    output_list = [remove_from_string(item) for item in input_list]
    
    return output_list

In [9]:
output_list = remove_keywords(df_lastline)
len(output_list)
output_list

['The tweet appears to be a legitimate warning about a potential virus, and it does not contain any harmful or offensive language. The tone is informative and neutral, and the language used is clear and concise. The tweet does not contain any threats, harassment, or hate speech. It also does not promote any disinformation or misinformation. Therefore, based on the analysis, I would mark this tweet as "0", indicating that it is not harmful to society.',
 'Overall, the tweet appears to be a factual and neutral statement about the cancellation of anti-epidemic measures, and therefore, I mark it as 0, indicating that it is not harmful to society.',
 "The tweet is promoting a legitimate cause and providing information about the EU's efforts to address a global issue. The hashtags used are relevant and do not contain any harmful or offensive content. Overall, the tweet appears to be a positive and informative message that is not harmful to society.",
 'Overall, the tweet does not appear to b

In [20]:
def extract_integers(input_list):
    # This will store the extracted integers
    integers = []
    # This will store the index and sentences with more than one integer and those without keywords
    special_cases = {}
    
    # Define a regex pattern to match integers
    integer_pattern = re.compile(r'\b\d+\b')
    
    for index, item in enumerate(input_list):
 # Skip to the next item since this case is handled
        
        # Find all integers in the string item
        found_integers = integer_pattern.findall(item)
        
        if found_integers:
            if len(found_integers) > 1:
                special_cases[index] = item
                integers.append(2)
            # Convert found integers to int
            else:
                found_integers = list(map(int, found_integers))
           
            # Add the found integers to the result list
                integers.extend(found_integers)
            
            # Check if there are more than one integer and store the element if true

        else:
            # Check for keywords if no integers are found
            item_lower = item.lower()
            if 'true' in item_lower or 'accurate' in item_lower or 'factual':
                integers.append(0)
            elif 'false' in item_lower:
                integers.append(1)
            else:
                # Store the index and sentence if neither integers nor keywords are found
                integers.append(2)
                #special_cases[index] = item
    
    return integers, special_cases

In [21]:
labels_parsed, special_cases = extract_integers(output_list)
len(labels_parsed), len(special_cases)

(3693, 357)

In [22]:
special_cases

{27: "The tweet appears to be a legitimate and informative message from the European Union (EU) about a new program aimed at strengthening healthcare systems in Europe and preparing them for future crises such as the COVID-19 pandemic. The language used is neutral and objective, and the tone is professional and informative. There is no evidence of harmful or offensive content, such as hate speech, disinformation, or cyberbullying. The tweet does not contain any explicit threats or violent language, and it does not promote discrimination or harm towards any individual or group. Overall, the tweet appears to be a positive and constructive message that is intended to educate and inform the public about the EU's efforts to address the COVID-19 pandemic.",
 44: 'The tweet appears to be a genuine announcement from the European Economic and Social Committee (EESC) about a prize for projects that combat COVID-19 and its consequences. The language used is formal and informative, and the tone is

In [23]:
labels_parsed

[0,
 0,
 0,
 0,
 19,
 0,
 19,
 0,
 0,
 19,
 1,
 365,
 19,
 19,
 19,
 1,
 0,
 19,
 0,
 19,
 0,
 0,
 19,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 19,
 1,
 0,
 19,
 0,
 19,
 19,
 19,
 0,
 19,
 19,
 1,
 2,
 19,
 0,
 19,
 19,
 0,
 2,
 19,
 19,
 0,
 0,
 19,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 2,
 0,
 2,
 19,
 19,
 19,
 0,
 0,
 19,
 0,
 0,
 19,
 19,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 19,
 19,
 0,
 19,
 0,
 0,
 19,
 19,
 1,
 0,
 1,
 2,
 1,
 0,
 19,
 0,
 0,
 1,
 19,
 19,
 0,
 0,
 0,
 19,
 19,
 0,
 0,
 0,
 19,
 0,
 19,
 1,
 19,
 0,
 1,
 19,
 19,
 0,
 0,
 0,
 0,
 19,
 0,
 0,
 19,
 0,
 1,
 19,
 0,
 0,
 1,
 19,
 19,
 2,
 19,
 0,
 19,
 0,
 19,
 0,
 19,
 1,
 19,
 0,
 0,
 19,
 0,
 0,
 19,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 19,
 1,
 0,
 1,
 19,
 19,
 19,
 0,
 0,
 0,
 19,
 0,
 0,
 19,
 2,
 0,
 19,
 19,
 1,
 19,
 0,
 1,
 19,
 0,
 0,
 19,
 19,
 2,
 0,
 19,
 0,
 2,
 0,
 19,
 19,
 0,
 19,
 19,
 19,
 19,
 19,
 0,
 19,
 0,
 19,
 19,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 19,
 0,
 19,
 1,
 2,
 0,
 0,
 1,
 19,
 19,
 0,
 19,
 0,
 0,
 0,


In [24]:
df_gt

,tweet_text,class_label
0,"За тия, дет си мислят, че няма вирус https://t...",0
1,"Отменят се част от противоепидемичните мерки, ...",0
2,Oпазването на биоразнообразието 🦋🐅🐘🌴 остава пр...,0
3,Кратък обзор над мерките в сградостроителствот...,0
4,Нова платформа за споделяне на добри практики ...,0
...,...,...
3688,"Два случая с COVID-19 в СБАЛББ Троян, един в Л...",1
3689,Генералният секретар на Съвета на Европа Мария...,0
3690,Нов потвърден случай на Covid-19 в Шипково htt...,1
3691,"@Anne_Hidalgo @Paris И, въпреки това, вчера, з...",0


In [25]:

gold_ans = df_gt['class_label'].tolist()
print("gold_ans:", gold_ans)


gold_ans: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [26]:
k = [el for el in labels_parsed]
print(k)

[0, 0, 0, 0, 19, 0, 19, 0, 0, 19, 1, 365, 19, 19, 19, 1, 0, 19, 0, 19, 0, 0, 19, 0, 0, 0, 0, 2, 0, 0, 0, 0, 19, 1, 0, 19, 0, 19, 19, 19, 0, 19, 19, 1, 2, 19, 0, 19, 19, 0, 2, 19, 19, 0, 0, 19, 0, 0, 0, 0, 0, 0, 19, 2, 0, 2, 19, 19, 19, 0, 0, 19, 0, 0, 19, 19, 0, 0, 0, 0, 0, 19, 0, 19, 19, 0, 19, 0, 0, 19, 19, 1, 0, 1, 2, 1, 0, 19, 0, 0, 1, 19, 19, 0, 0, 0, 19, 19, 0, 0, 0, 19, 0, 19, 1, 19, 0, 1, 19, 19, 0, 0, 0, 0, 19, 0, 0, 19, 0, 1, 19, 0, 0, 1, 19, 19, 2, 19, 0, 19, 0, 19, 0, 19, 1, 19, 0, 0, 19, 0, 0, 19, 0, 0, 0, 2, 0, 0, 0, 0, 19, 1, 0, 1, 19, 19, 19, 0, 0, 0, 19, 0, 0, 19, 2, 0, 19, 19, 1, 19, 0, 1, 19, 0, 0, 19, 19, 2, 0, 19, 0, 2, 0, 19, 19, 0, 19, 19, 19, 19, 19, 0, 19, 0, 19, 19, 0, 0, 0, 0, 0, 19, 0, 19, 0, 19, 1, 2, 0, 0, 1, 19, 19, 0, 19, 0, 0, 0, 19, 19, 19, 19, 19, 19, 0, 0, 0, 0, 0, 0, 19, 0, 0, 19, 19, 0, 0, 0, 0, 19, 1, 0, 19, 1, 0, 0, 2, 19, 0, 0, 0, 0, 0, 0, 0, 19, 1, 0, 0, 19, 19, 19, 0, 0, 0, 0, 19, 0, 1, 0, 19, 19, 2, 0, 0, 0, 19, 19, 1, 0, 19, 19, 19, 19, 0, 0

In [27]:
def compute_metrics(predictions, labels):
        accuracy_metric = evaluate.load("accuracy")
        precision_metric = evaluate.load("precision")
        recall_metric = evaluate.load("recall")
        f1_metric = evaluate.load("f1")


        accuracy = accuracy_metric.compute(predictions=predictions, references=labels)[
            "accuracy"
        ]
        precision = precision_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["precision"]
        recall = recall_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["recall"]
        f1 = f1_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["f1"]

        return {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1-score": f1,
        }


results=compute_metrics(labels_parsed, gold_ans)
print(f"FNNet, CoT, prompt with temperature=0.1:\n", json.dumps(results, indent=4))

FNNet, CoT, prompt with temperature=0.1:
 {
    "accuracy": 0.40942323314378554,
    "precision": 0.795064098479279,
    "recall": 0.40942323314378554,
    "f1-score": 0.5396120733633354
}


/home/michele.maggini/miniconda3/envs/bert_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
